<a href="https://colab.research.google.com/github/sudeepmenon17/PytorchforNN/blob/main/WT_for_VIV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

!pip install google-auth
from google.colab import auth
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
#!ls /content/drive/MyDrive/

 Aadhar_lakshmi.docx				     model_test.png
 Aadhar_sudeep.docx				     my_model.onnx
 airfoil_model.keras				    'New Doc 2019-01-09 20.58.00.pdf'
 airfoil_model_tf				    'New Doc 2019-05-06 19.36.52.pdf'
 airfoil_model_tf_final				    'New Doc 2019-05-10 23.45.12_1.jpg'
 aoa14_beta_predicted_tecplot.dat		    'New Doc 2019-05-10 23.45.12_2.jpg'
 aoa14_beta_tecplot.dat				    'New Doc 2019-05-10 23.45.12.pdf'
 best_model.keras				     Passport_lakshmi.docx
 beta11.txt					     Passport_sudeep.docx
 beta1.txt					    'passprt (1).pdf'
 beta22.txt					     passprt.pdf
 beta2.txt					     pytorch_model_karim_converted.pt
 beta33.txt					     pytorch_model_karim.onnx
 beta3.txt					     pytorch_model_karim.pth
 beta44.txt					     Re500_RV5_D0p9
 beta4.txt					     Resume.pdf
 CheckInReceipt.pdf				     RESUME.pdf
'Colab Notebooks'				     SKMBT_C224e16011411180.pdf
 data_set1.csv					     SSR_TSRPT.pdf
 data_set2.csv					     starccm_report.csv
 data_set3.csv					    'Sudeep Menon-1456-A

In [12]:
import numpy as np
from scipy.io import loadmat, savemat

# Load the .mat files
vor_data = loadmat('/content/drive/MyDrive/Re500_RV5_D0p9/VORTALL.mat')
time_data = loadmat('/content/drive/MyDrive/Re500_RV5_D0p9/Time.mat')
points_data = loadmat('/content/drive/MyDrive/Re500_RV5_D0p9/points.mat')

# Extract necessary variables
VORTALL = vor_data['VORTALL']  # Vorticity data
Time = time_data['Time'].flatten()  # Time vector (flatten to 1D)
points = points_data['points']  # Points array


In [13]:
# Transformation parameters
t0 = 400            # Reference time step
A = 0.5             # Amplitude of sinusoidal motion
omega = 0.2513    # Frequency of oscillation (RV = 5)
y0 = A * np.sin(t0 * omega)  # Baseline vertical offset

# Extract x and y coordinates of points
px = points[:, 0]  # x-coordinates
py = points[:, 1]  # y-coordinates

In [16]:
# Initialize new y-coordinates for each time step
py_transformed = []

# Apply the sinusoidal transformation for each time step
for t in Time:
    py_new = py + A * np.sin(t * omega) - y0
    py_transformed.append(py_new)

# Convert the list to a numpy array for consistency
py_transformed = np.array(py_transformed).T  # Transpose to match the MATLAB output shape


output_data = {
    'px': px,  # Original x-coordinates (unchanged)
    'py_transformed': py_transformed,  # Transformed y-coordinates
    'Time': Time  # Time array
}
savemat('/content/drive/MyDrive/Re500_RV5_D0p9/Ground_data.mat', output_data)

print("Transformation complete. Transformed data saved to 'Ground_data.mat'.")

Transformation complete. Transformed data saved to 'Ground_data.mat'.


In [30]:
!pip install PyWavelets
import pywt
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [ ]:
# Load the transformed ground data
data = loadmat('/content/drive/MyDrive/Re500_RV5_D0p9/Ground_data.mat')

# Extract relevant data
px = data['px'].flatten()  # x-coordinates
py_transformed = data['py_transformed']  # Transformed y-coordinates
Time = data['Time'].flatten()  # Time array

# Create a grid for visualization
nx = len(np.unique(px))  # Assume regular grid
ny = py_transformed.shape[0] // nx
x_grid = px.reshape(ny, nx)
y_grid = py_transformed[:, 0].reshape(ny, nx)  # Use the initial time step as a reference

# Initialize the figure
fig, ax = plt.subplots()
cax = ax.pcolormesh(x_grid, y_grid, py_transformed[:, 0].reshape(ny, nx), cmap='viridis')
fig.colorbar(cax, ax=ax)
title = ax.set_title("Time = {:.2f}".format(Time[0]))

def update(frame):
    ax.clear()
    cax = ax.pcolormesh(
        x_grid, y_grid, py_transformed[:, frame].reshape(ny, nx), cmap='viridis'
    )
    ax.set_title("Time = {:.2f}".format(Time[frame]))
    return cax
# Create the animation
ani = animation.FuncAnimation(fig, update, frames=len(Time), interval=100, repeat=True)

# Save and display animation in HTML format
HTML(ani.to_jshtml())


<ipython-input-32-a590b232e46d>:17: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  cax = ax.pcolormesh(x_grid, y_grid, py_transformed[:, 0].reshape(ny, nx), cmap='viridis')
<ipython-input-32-a590b232e46d>:23: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  cax = ax.pcolormesh(
